In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout
from tensorflow.keras.optimizers import Adam

# CIFAR-10 데이터셋을 읽고 신경망에 입력할 형태로 변환
(x_train,y_train),(x_test,y_test)=cifar10.load_data()
x_train=x_train.astype(np.float32)/255.0
x_test=x_test.astype(np.float32)/255.0
y_train=tf.keras.utils.to_categorical(y_train,10)
y_test=tf.keras.utils.to_categorical(y_test,10)

# 신경망 모델 설계
cnn=Sequential()
cnn.add(Conv2D(32,(3,3),activation='relu',input_shape=(32,32,3)))
cnn.add(Conv2D(32,(3,3),activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))
cnn.add(Conv2D(64,(3,3),activation='relu'))
cnn.add(Conv2D(64,(3,3),activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))
cnn.add(Flatten())
cnn.add(Dense(512,activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(10,activation='softmax'))

# 신경망 모델 학습
cnn.compile(loss='categorical_crossentropy',optimizer=Adam(),metrics=['accuracy'])
hist=cnn.fit(x_train,y_train,batch_size=128,epochs=30,validation_data=(x_test,y_test),verbose=2)

cnn.summary() # cnn 모델의 정보 출력

for layer in cnn.layers: # 컨볼루션층의 커널 시각화
    if 'conv' in layer.name:
        kernel,biases=layer.get_weights()
        print(layer.name,kernel.shape) # 커널의 텐서 모양 출력

kernel,biases=cnn.layers[0].get_weights() # 층 0의 커널 정보 저장
minv,maxv=kernel.min(),kernel.max()
kernel=(kernel-minv)/(maxv-minv)
n_kernel=32

import matplotlib.pyplot as plt

plt.figure(figsize=(20,3))
plt.suptitle("Kernels of conv2d_4")
for i in range(n_kernel):
    f=kernel[:,:,:,i]
    for j in range(3):
        plt.subplot(3,n_kernel,j*n_kernel+i+1)
        plt.imshow(f[:,:,j],cmap='gray')
        plt.xticks([]); plt.yticks([])
        plt.title(str(i)+'_'+str(j))
plt.show()

for layer in cnn.layers:
    if 'conv' in layer.name:
        print(layer.name,layer.output.shape)

from tensorflow.keras.models import Model

partial_model=Model(inputs=cnn.inputs,outputs=cnn.layers[0].output) # 층 0만 떼어냄
partial_model.summary()

feature_map=partial_model.predict(x_test) # 부분 모델로 테스트 집합을 예측
fm=feature_map[1]

plt.imshow(x_test[1])

plt.figure(figsize=(20,3))
plt.suptitle("Feature maps of conv2d_4")
for i in range(32):
    plt.subplot(2,16,i+1)
    plt.imshow(fm[:,:,i],cmap='gray')
    plt.xticks([]); plt.yticks([])
    plt.title("map"+str(i))
plt.show()

Epoch 1/30
391/391 - 12s - 30ms/step - accuracy: 0.4185 - loss: 1.5968 - val_accuracy: 0.5709 - val_loss: 1.2033
Epoch 2/30
391/391 - 4s - 10ms/step - accuracy: 0.5899 - loss: 1.1472 - val_accuracy: 0.6683 - val_loss: 0.9356
Epoch 3/30
391/391 - 4s - 10ms/step - accuracy: 0.6600 - loss: 0.9625 - val_accuracy: 0.6990 - val_loss: 0.8638
Epoch 4/30
391/391 - 4s - 10ms/step - accuracy: 0.7008 - loss: 0.8484 - val_accuracy: 0.7304 - val_loss: 0.7641
Epoch 5/30
391/391 - 5s - 13ms/step - accuracy: 0.7317 - loss: 0.7624 - val_accuracy: 0.7412 - val_loss: 0.7348
Epoch 6/30
391/391 - 5s - 13ms/step - accuracy: 0.7534 - loss: 0.7022 - val_accuracy: 0.7657 - val_loss: 0.6751
Epoch 7/30
391/391 - 4s - 10ms/step - accuracy: 0.7715 - loss: 0.6494 - val_accuracy: 0.7754 - val_loss: 0.6599
Epoch 8/30
391/391 - 4s - 10ms/step - accuracy: 0.7859 - loss: 0.6079 - val_accuracy: 0.7664 - val_loss: 0.6773
Epoch 9/30
391/391 - 4s - 10ms/step - accuracy: 0.7988 - loss: 0.5698 - val_accuracy: 0.7764 - val_loss